# Search Output

This notebook generates json files for searching through ChIP-pro's genes and TFs.

In [1]:
import os
import pandas as pd
import glob

## TFs

In [2]:
prefix = '../data/'
suffix = '/TF_list.csv'

organisms = [f.name for f in os.scandir(prefix)]
cols_of_interest = ['TF', 'Organism', 'Strain', 'Media', 'Supplement']

all_tfs = pd.DataFrame(columns = cols_of_interest + ['link'])

for o in organisms:
    try:
        tfs = pd.read_csv(prefix + o + suffix, index_col = 0)
    except NotADirectoryError:
        continue
        
    # build link
    for i in tfs.index:
        l = 'tf_dashboard.html?organism=' + o
        l += '&tf=' + tfs.TF[i]
        l += '&genome=' + tfs.genome_id[i]
        l += '&i=' + str(i)
        tfs.loc[i, 'link'] = l
    
    # take info of interest
    tfs = tfs[cols_of_interest + ['link']]
    
    # save
    all_tfs = all_tfs.append(tfs, ignore_index = True)
    
all_tfs

,TF,Organism,Strain,Media,Supplement,link
0,CodY,Staphylococcus aureus,LAC chromosome,NaN,NaN,tf_dashboard.html?organism=s_aureus&tf=CodY&ge...
1,SigS,Staphylococcus aureus,USA300_TCH1516,NaN,NaN,tf_dashboard.html?organism=s_aureus&tf=SigS&ge...
2,VraR,Staphylococcus aureus,USA300_TCH1516,NaN,NaN,tf_dashboard.html?organism=s_aureus&tf=VraR&ge...
3,Fur,Shigella flexneri,"2a, 2457T",NaN,NaN,tf_dashboard.html?organism=s_flexneri&tf=Fur&g...
4,Fur,Klebsiella pneumoniae,MGH 78578,NaN,NaN,tf_dashboard.html?organism=k_pneumoniae&tf=Fur...
...,...,...,...,...,...,...
62,Fur,Pseudomonas putida,KT2440,NaN,NaN,tf_dashboard.html?organism=p_putida&tf=Fur&gen...
63,Sigma D,Pseudomonas putida,KT2440,NaN,NaN,tf_dashboard.html?organism=p_putida&tf=Sigma D...
64,Fur,Yersinia pseudotuberculosis,IP 32953,NaN,NaN,tf_dashboard.html?organism=y_pseudotuberculosi...
65,Fur,Salmonella enterica,Typhimurium LT2,NaN,Fe,tf_dashboard.html?organism=s_enterica&tf=Fur&g...


In [3]:
all_tfs.to_json('../data/tfs_search.json', orient='records')

## Genes

In [4]:
prefix = '../data/'
suffix = '_binding_table.json'

all_genes = pd.DataFrame(columns = ['Gene', 'Locus', 
        'Organism', 'Strain', 'Binding_site_id',
        'Condition', 'Peak_start', 'Peak_end',
        'Peak_strength', 'link'])

for j, tf_row in all_tfs.iterrows():
    
    # get folder info from the link & read in file
    file = tf_row.link.replace('&', '=').split('=')
    path = prefix + file[1] + '/' + file[5] + '/table/' + file[3].lower() + "_*binding_table.json"
    all_matches = glob.glob(path)
    
    for f in all_matches:
        # read in
        try:
            bt = pd.read_json(f)
        except (FileNotFoundError, ValueError) as e:
            continue

        for i, row in bt.iterrows():

            # get genes & loci
            genes = row.target_genes.split(',')
            loci = row.target_locus.split(',')

            # rearrange data for new table
            for g, l in zip(genes, loci):
                if g == '':
                    continue
                all_genes = all_genes.append({
                    'Gene':g,
                    'Locus':l,
                    'Organism':tf_row.Organism,
                    'Strain':tf_row.Strain,
                    'Binding_site_id':row['index'],
                    'Condition':row.condition,
                    'Peak_start':row.binding_peak_start,
                    'Peak_end':row.binding_peak_end,
                    'Peak_strength':row.binding_peak_strength,
                    'link':tf_row.link}, ignore_index = True)
            
all_genes = all_genes.sort_values('Peak_start')

In [5]:
all_genes

,Gene,Locus,Organism,Strain,Binding_site_id,Condition,Peak_start,Peak_end,Peak_strength,link
37809,PP_0001,PP_0001,Pseudomonas putida,KT2440,Sigma D-1,sigma d + M9,43,82,8.09,tf_dashboard.html?organism=p_putida&tf=Sigma D...
28975,thrL,b0001,Escherichia coli,K-12 MG1655,Rob-1,rob + expo,138,168,3.05,tf_dashboard.html?organism=e_coli&tf=Rob&genom...
28976,thrA,b0002,Escherichia coli,K-12 MG1655,Rob-1,rob + expo,138,168,3.05,tf_dashboard.html?organism=e_coli&tf=Rob&genom...
28977,thrB,b0003,Escherichia coli,K-12 MG1655,Rob-1,rob + expo,138,168,3.05,tf_dashboard.html?organism=e_coli&tf=Rob&genom...
28978,thrC,b0004,Escherichia coli,K-12 MG1655,Rob-1,rob + expo,138,168,3.05,tf_dashboard.html?organism=e_coli&tf=Rob&genom...
...,...,...,...,...,...,...,...,...,...,...
38526,PP_5393,PP_5393,Pseudomonas putida,KT2440,Sigma D-718,sigma d + M9,6147517,6147549,5.60,tf_dashboard.html?organism=p_putida&tf=Sigma D...
38527,PP_5395,PP_5395,Pseudomonas putida,KT2440,Sigma D-719,sigma d + M9,6150914,6150950,5.91,tf_dashboard.html?organism=p_putida&tf=Sigma D...
38528,PP_5399,PP_5399,Pseudomonas putida,KT2440,Sigma D-720,sigma d + M9,6156165,6156205,1.28,tf_dashboard.html?organism=p_putida&tf=Sigma D...
38529,PP_5739,PP_5739,Pseudomonas putida,KT2440,Sigma D-721,sigma d + M9,6158907,6158922,6.74,tf_dashboard.html?organism=p_putida&tf=Sigma D...


In [6]:
all_genes.to_json('../data/genes_search.json', orient='records')